### Disclaimer

This software is provided solely for educational and research purposes. 
It is not intended to provide investment advice, and no investment recommendations are made herein. 
The developers are not financial advisors and accept no responsibility for any financial decisions or losses resulting from the use of this software. 
Always consult a professional financial advisor before making any investment decisions.

### Imports

In [1]:
import yfinance as yf
from datetime import datetime, timedelta
from scipy.interpolate import interp1d
import numpy as np
import threading

### Functions

In [2]:
def filter_dates(dates):
    today = datetime.today().date()
    cutoff_date = today + timedelta(days=45)
    
    sorted_dates = sorted(datetime.strptime(date, "%Y-%m-%d").date() for date in dates)

    arr = []
    for i, date in enumerate(sorted_dates):
        if date >= cutoff_date:
            arr = [d.strftime("%Y-%m-%d") for d in sorted_dates[:i+1]]  
            break
    
    if len(arr) > 0:
        if arr[0] == today.strftime("%Y-%m-%d"):
            return arr[1:]
        return arr

    raise ValueError("No date 45 days or more in the future found.")


def yang_zhang(price_data, window=30, trading_periods=252, return_last_only=True):
    log_ho = (price_data['High'] / price_data['Open']).apply(np.log)
    log_lo = (price_data['Low'] / price_data['Open']).apply(np.log)
    log_co = (price_data['Close'] / price_data['Open']).apply(np.log)
    
    log_oc = (price_data['Open'] / price_data['Close'].shift(1)).apply(np.log)
    log_oc_sq = log_oc**2
    
    log_cc = (price_data['Close'] / price_data['Close'].shift(1)).apply(np.log)
    log_cc_sq = log_cc**2
    
    rs = log_ho * (log_ho - log_co) + log_lo * (log_lo - log_co)
    
    close_vol = log_cc_sq.rolling(
        window=window,
        center=False
    ).sum() * (1.0 / (window - 1.0))

    open_vol = log_oc_sq.rolling(
        window=window,
        center=False
    ).sum() * (1.0 / (window - 1.0))

    window_rs = rs.rolling(
        window=window,
        center=False
    ).sum() * (1.0 / (window - 1.0))

    k = 0.3333 / (1.3333 + ((window + 1) / (window - 1)) )
    result = (open_vol + k * close_vol + (1 - k) * window_rs).apply(np.sqrt) * np.sqrt(trading_periods)

    if return_last_only:
        return result.iloc[-1]
    else:
        return result.dropna()

def build_term_structure(days, ivs):
    days = np.array(days)
    ivs = np.array(ivs)

    sort_idx = days.argsort()
    days = days[sort_idx]
    ivs = ivs[sort_idx]

    spline = interp1d(days, ivs, kind='linear', fill_value="extrapolate")

    def term_spline(dte):
        if dte < days[0]:  
            return ivs[0]
        elif dte > days[-1]:
            return ivs[-1]
        else:  
            return float(spline(dte))
    return term_spline

def get_current_price(df_price_history):
    return df_price_history['Close'].iloc[-1]


def calc_kelly_bet(
    p_win: float = 0.66,
    odds_decimal: float = 1.66,
    current_bankroll: float = 10000,
    pct_kelly=0.10
) -> float:
    """
    Calculates the Kelly Criterion optimal bet amount.

    The Kelly Criterion is a formula used to determine the optimal size of a series
    of bets to maximize the long-term growth rate of a bankroll.

    Args:
        p_win: The estimated probability of winning the bet (p),
                                a float between 0 and 1.
        odds_decimal: The decimal odds (b), where a successful $1 bet returns $b.
                      For example, if odds are 2:1, odds_decimal is 3.0.
                      If odds are 1:1, odds_decimal is 2.0.
                      This is (payout / stake) + 1.
        current_bankroll: The total amount of money available to bet (B).

    Returns:
        The calculated optimal bet amount. Returns 0 if the bet is not favorable
        (i.e., the calculated fraction is negative or zero), or if inputs are invalid.
    """
    if not (0 <= p_win <= 1):
        raise ValueError("Probability of winning must be between 0 and 1.")
    if odds_decimal <= 1.0: # Odds must be greater than 1.0 (e.g., 1.01 for a tiny profit)
        raise ValueError("Decimal odds must be greater than 1.0 (e.g., 1.01 for a winning bet).")
    if current_bankroll <= 0:
        raise ValueError("Current bankroll must be a positive number.")

    b_kelly = odds_decimal - 1.0

    if b_kelly <= 0: # Should be caught by odds_decimal check, but as a safeguard
        return 0.0

    kelly_fraction = p_win - ((1 - p_win) / b_kelly)

    if kelly_fraction <= 0:
        return 0.0

    bet_amount = kelly_fraction * current_bankroll
    bet_amount = bet_amount * pct_kelly
    return round(bet_amount, 2)

def compute_recommendation(ticker, min_avg_30d_volume=1500000, min_iv30_rv30=2.0, max_ts_slope_0_45=-0.0075):
    try:
        ticker = ticker.strip().upper()
        if not ticker:
            return "No stock symbol provided."
        
        try:
            stock = yf.Ticker(ticker)
            if len(stock.options) == 0:
                raise KeyError()
        except KeyError:
            return f"Error: No options found for stock symbol '{ticker}'."
        
        exp_dates = list(stock.options)
        try:
            exp_dates = filter_dates(exp_dates)
        except:
            return "Error: Not enough option data."
        
        options_chains = {}
        for exp_date in exp_dates:
            options_chains[exp_date] = stock.option_chain(exp_date)

        df_price_history = stock.history(period='3mo')
        df_price_history = df_price_history.sort_index()
        
        try:
            underlying_price = get_current_price(df_price_history)
            if underlying_price is None:
                raise ValueError("No market price found.")
        except Exception:
            return "Error: Unable to retrieve underlying stock price."
        
        atm_iv = {}
        straddle = None 
        i = 0
        for exp_date, chain in options_chains.items():
            calls = chain.calls
            puts = chain.puts

            if calls.empty or puts.empty:
                continue

            call_diffs = (calls['strike'] - underlying_price).abs()
            call_idx = call_diffs.idxmin()
            call_iv = calls.loc[call_idx, 'impliedVolatility']

            put_diffs = (puts['strike'] - underlying_price).abs()
            put_idx = put_diffs.idxmin()
            put_iv = puts.loc[put_idx, 'impliedVolatility']

            atm_iv_value = (call_iv + put_iv) / 2.0
            atm_iv[exp_date] = atm_iv_value

            if i == 0:
                call_bid = calls.loc[call_idx, 'bid']
                call_ask = calls.loc[call_idx, 'ask']
                put_bid = puts.loc[put_idx, 'bid']
                put_ask = puts.loc[put_idx, 'ask']
                
                if call_bid is not None and call_ask is not None:
                    call_mid = (call_bid + call_ask) / 2.0
                else:
                    call_mid = None

                if put_bid is not None and put_ask is not None:
                    put_mid = (put_bid + put_ask) / 2.0
                else:
                    put_mid = None

                if call_mid is not None and put_mid is not None:
                    straddle = (call_mid + put_mid)

            i += 1
        
        if not atm_iv:
            return "Error: Could not determine ATM IV for any expiration dates."
        
        today = datetime.today().date()
        dtes = []
        ivs = []
        for exp_date, iv in atm_iv.items():
            exp_date_obj = datetime.strptime(exp_date, "%Y-%m-%d").date()
            days_to_expiry = (exp_date_obj - today).days
            dtes.append(days_to_expiry)
            ivs.append(iv)
        
        term_spline = build_term_structure(dtes, ivs)
        
        ts_slope_0_45 = (term_spline(45) - term_spline(dtes[0])) / (45-dtes[0])
        
        iv30_rv30 = term_spline(30) / yang_zhang(df_price_history)

        avg_volume = df_price_history['Volume'].rolling(30).mean().dropna().iloc[-1]

        expected_move = str(round(straddle / underlying_price * 100,2)) + "%" if straddle else None

        result_summary = {
            'avg_volume': round(avg_volume, 3),
            'avg_volume_pass': avg_volume >= min_avg_30d_volume,
            'iv30_rv30': round(iv30_rv30, 3),
            'iv30_rv30_pass': iv30_rv30 >= min_iv30_rv30,
            'ts_slope_0_45': ts_slope_0_45,
            'ts_slope_0_45_pass': ts_slope_0_45 <= max_ts_slope_0_45,
            'expected_move': expected_move
        }
        
        if result_summary['avg_volume_pass'] and result_summary['iv30_rv30_pass'] and result_summary['ts_slope_0_45_pass']:
            suggestion = "Recommended"
        elif result_summary['ts_slope_0_45_pass'] and ((result_summary['avg_volume_pass'] and not result_summary['iv30_rv30_pass']) or (result_summary['iv30_rv30_pass'] and not result_summary['avg_volume_pass'])):
            suggestion = "Consider"
        else:
            suggestion = "Avoid"

        result_summary["suggestion"] = suggestion

        if suggestion == "Recommended":
            kelly_bet = calc_kelly_bet()
        elif suggestion == "Consider":
            kelly_bet = calc_kelly_bet(p_win=0.33)
        else:
            kelly_bet = 0

        result_summary["kelly_bet"] = kelly_bet
        
        # Check that they are in our desired range (see video)
        return result_summary
    
    except Exception as e:
        raise Exception(f'Error occured processing')

In [3]:
df_price_history = yf.Ticker('AAPL').history(period='3mo')
df_price_history = df_price_history.sort_index()

### Rules to open and close the positions
- Enter within 15 minutes of the closing bell for same day earnings
- Exit within 15 minutes of market open the following day

In [26]:
compute_recommendation('NVDA')

{'avg_volume': np.float64(180522236.66666666),
 'avg_volume_pass': np.True_,
 'iv30_rv30': np.float64(3.1281325814570127),
 'iv30_rv30_pass': np.True_,
 'ts_slope_0_45': -0.01752149483816964,
 'ts_slope_0_45_pass': True,
 'expected_move': '79.9%',
 'suggestion': 'Recommended',
 'kelly_bet': 1448.484848484849}

# Line by line

#### params

In [5]:
ticker = "AAPL"
min_avg_30d_volume=1500000
min_iv30_rv30=2.0
max_ts_slope_0_45=-0.0075

#### ticker initiation

In [8]:
ticker = ticker.strip().upper()
if not ticker:
    raise "No stock symbol provided."

try:
    stock = yf.Ticker(ticker)
    if len(stock.options) == 0:
        raise KeyError()
except KeyError:
    raise f"Error: No options found for stock symbol '{ticker}'."

stock

yfinance.Ticker object <AAPL>

#### get expiration dates

In [9]:
exp_dates = list(stock.options)
print(f"orig: {exp_dates}")
try:
    exp_dates = filter_dates(exp_dates)
    print(f"filtered: {exp_dates}")
except:
    raise "Error: Not enough option data."

orig: ['2025-07-25', '2025-08-01', '2025-08-08', '2025-08-15', '2025-08-22', '2025-08-29', '2025-09-19', '2025-10-17', '2025-11-21', '2025-12-19', '2026-01-16', '2026-02-20', '2026-03-20', '2026-05-15', '2026-06-18', '2026-09-18', '2026-12-18', '2027-01-15', '2027-06-17', '2027-12-17']
filtered: ['2025-07-25', '2025-08-01', '2025-08-08', '2025-08-15', '2025-08-22', '2025-08-29', '2025-09-19']


#### get option chain data

In [11]:
%%time

options_chains = {}
for exp_date in exp_dates:
    options_chains[exp_date] = stock.option_chain(exp_date)
options_chains

CPU times: user 80.4 ms, sys: 9.51 ms, total: 89.9 ms
Wall time: 711 ms


{'2025-07-25': Options(calls=         contractSymbol             lastTradeDate  strike  lastPrice     bid  \
 0   AAPL250725C00110000 2025-07-17 19:37:42+00:00   110.0     101.07  100.70   
 1   AAPL250725C00120000 2025-06-27 17:42:44+00:00   120.0      81.97   90.70   
 2   AAPL250725C00125000 2025-07-17 14:34:08+00:00   125.0      85.20   85.70   
 3   AAPL250725C00135000 2025-06-23 17:28:40+00:00   135.0      67.00   75.65   
 4   AAPL250725C00140000 2025-07-18 14:13:48+00:00   140.0      70.15   70.80   
 5   AAPL250725C00145000 2025-06-24 16:49:05+00:00   145.0      58.59   65.75   
 6   AAPL250725C00150000 2025-07-18 19:32:02+00:00   150.0      61.00   60.60   
 7   AAPL250725C00155000 2025-07-16 14:20:50+00:00   155.0      57.38   55.90   
 8   AAPL250725C00160000 2025-07-18 13:57:24+00:00   160.0      51.17   51.00   
 9   AAPL250725C00165000 2025-07-17 15:02:21+00:00   165.0      45.45   45.75   
 10  AAPL250725C00170000 2025-07-17 15:02:21+00:00   170.0      40.45   40.70   


#### underlying price history (3mo)

In [12]:
df_price_history = stock.history(period='3mo')
df_price_history = df_price_history.sort_index()
df_price_history.head(2)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2025-04-21 00:00:00-04:00,193.016885,193.546189,189.561409,192.907028,46742500,0.0,0.0
2025-04-22 00:00:00-04:00,195.863154,201.325992,195.713357,199.478424,52976400,0.0,0.0


#### current price

In [13]:
try:
    underlying_price = get_current_price(df_price_history)
    if underlying_price is None:
        raise ValueError("No market price found.")
except Exception:
    raise "Error: Unable to retrieve underlying stock price."

underlying_price

np.float64(211.17999267578125)

#### IV Calculation (ATM)

In [17]:
options_chains['2025-07-25'].calls

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,AAPL250725C00110000,2025-07-17 19:37:42+00:00,110.0,101.07,100.70,102.05,0.000000,0.000000,27.0,51,2.492191,True,REGULAR,USD
1,AAPL250725C00120000,2025-06-27 17:42:44+00:00,120.0,81.97,90.70,92.10,0.000000,0.000000,5.0,5,2.226567,True,REGULAR,USD
2,AAPL250725C00125000,2025-07-17 14:34:08+00:00,125.0,85.20,85.70,87.15,0.000000,0.000000,10.0,1,2.113286,True,REGULAR,USD
3,AAPL250725C00135000,2025-06-23 17:28:40+00:00,135.0,67.00,75.65,77.15,0.000000,0.000000,2.0,1,1.806642,True,REGULAR,USD
4,AAPL250725C00140000,2025-07-18 14:13:48+00:00,140.0,70.15,70.80,71.75,-0.180000,-0.255937,1.0,10,1.496096,True,REGULAR,USD
5,AAPL250725C00145000,2025-06-24 16:49:05+00:00,145.0,58.59,65.75,66.75,0.000000,0.000000,1.0,1,1.328128,True,REGULAR,USD
6,AAPL250725C00150000,2025-07-18 19:32:02+00:00,150.0,61.00,60.60,61.75,0.509998,0.843112,1.0,71,1.674806,True,REGULAR,USD
7,AAPL250725C00155000,2025-07-16 14:20:50+00:00,155.0,57.38,55.90,56.75,0.000000,0.000000,1.0,3,1.226566,True,REGULAR,USD
8,AAPL250725C00160000,2025-07-18 13:57:24+00:00,160.0,51.17,51.00,51.65,-0.320003,-0.621487,44.0,6329,1.115239,True,REGULAR,USD
9,AAPL250725C00165000,2025-07-17 15:02:21+00:00,165.0,45.45,45.75,46.80,0.000000,0.000000,1.0,41,0.949219,True,REGULAR,USD


#### loop over dict where key is exp_date and value is option chain df
- need to add assertion that the last trade timestamp > market close timestamp - 15 minutes

In [21]:
call_diffs = (options_chains['2025-07-25'].calls['strike'] - underlying_price).abs()
call_idx = call_diffs.idxmin()
print(call_idx)

call_diffs[0:3]

23


0    101.179993
1     91.179993
2     86.179993
Name: strike, dtype: float64

In [55]:
call_bid, call_ask, put_bid, put_ask

(np.float64(3.25), np.float64(3.35), np.float64(1.89), np.float64(1.95))

In [50]:
atm_iv = {}
straddle = None 
i = 0
for exp_date, chain in options_chains.items():
    calls = chain.calls
    puts = chain.puts

    if calls.empty or puts.empty:
        continue

    # get the closest strike to the underlying --> so we can get the ATM straddle with the associated IV
    call_diffs = (calls['strike'] - underlying_price).abs()
    call_idx = call_diffs.idxmin()
    call_iv = calls.loc[call_idx, 'impliedVolatility']

    put_diffs = (puts['strike'] - underlying_price).abs()
    put_idx = put_diffs.idxmin()
    put_iv = puts.loc[put_idx, 'impliedVolatility']

    atm_iv_value = (call_iv + put_iv) / 2.0
    atm_iv[exp_date] = atm_iv_value

    # get midpoint of bid/ask spread for both calls and puts
    if i == 0:
        call_bid = calls.loc[call_idx, 'bid']
        call_ask = calls.loc[call_idx, 'ask']
        put_bid = puts.loc[put_idx, 'bid']
        put_ask = puts.loc[put_idx, 'ask']
        
        if call_bid is not None and call_ask is not None:
            call_mid = (call_bid + call_ask) / 2.0
        else:
            call_mid = None

        if put_bid is not None and put_ask is not None:
            put_mid = (put_bid + put_ask) / 2.0
        else:
            put_mid = None

        if call_mid is not None and put_mid is not None:
            straddle = (call_mid + put_mid)

    i += 1

if not atm_iv:
    raise "Error: Could not determine ATM IV for any expiration dates."

#### get dtes and IV for ATM straddles

In [25]:
today = datetime.today().date()
dtes = []
ivs = []
for exp_date, iv in atm_iv.items():
    exp_date_obj = datetime.strptime(exp_date, "%Y-%m-%d").date()
    days_to_expiry = (exp_date_obj - today).days
    dtes.append(days_to_expiry)
    ivs.append(iv)

print(dtes)
print(ivs)

[5, 12, 19, 26, 33, 40, 61]
[np.float64(0.2652661364746094), np.float64(0.3581607153320312), np.float64(0.3265448205566406), np.float64(0.30823445678710926), np.float64(0.30136807037353514), np.float64(0.29264013031005853), np.float64(0.2756877851867676)]


#### build term structure

In [46]:
term_spline = build_term_structure(dtes, ivs)
ts_slope_0_45 = (term_spline(45) - term_spline(dtes[0])) / (45-dtes[0])
iv30_rv30 = term_spline(30) / yang_zhang(df_price_history)
avg_volume = df_price_history['Volume'].rolling(30).mean().dropna().iloc[-1]
expected_move = str(round(straddle / underlying_price * 100,2)) + "%" if straddle else None
result_summary = {
    'avg_volume': round(avg_volume, 3),
    'avg_volume_pass': avg_volume >= min_avg_30d_volume,
    'iv30_rv30': round(iv30_rv30, 3),
    'iv30_rv30_pass': iv30_rv30 >= min_iv30_rv30,
    'ts_slope_0_45': ts_slope_0_45,
    'ts_slope_0_45_pass': ts_slope_0_45 <= max_ts_slope_0_45,
    'expected_move': expected_move
}

if result_summary['avg_volume_pass'] and result_summary['iv30_rv30_pass'] and result_summary['ts_slope_0_45_pass']:
    suggestion = "Recommended"
elif result_summary['ts_slope_0_45_pass'] and ((result_summary['avg_volume_pass'] and not result_summary['iv30_rv30_pass']) or (result_summary['iv30_rv30_pass'] and not result_summary['avg_volume_pass'])):
    suggestion = "Consider"
else:
    suggestion = "Avoid"

result_summary["suggestion"] = suggestion

if suggestion == "Recommended":
    kelly_bet = calc_kelly_bet()
elif suggestion == "Consider":
    kelly_bet = calc_kelly_bet(p_win=0.33)
else:
    kelly_bet = 0

result_summary["kelly_bet"] = kelly_bet

In [47]:
result_summary

{'avg_volume': np.float64(53611936.667),
 'avg_volume_pass': np.True_,
 'iv30_rv30': np.float64(1.383),
 'iv30_rv30_pass': np.False_,
 'ts_slope_0_45': 0.0005834430296761631,
 'ts_slope_0_45_pass': False,
 'expected_move': '2.47%',
 'suggestion': 'Avoid',
 'kelly_bet': 0}